In [1]:
import random, numpy as np, pandas as pd
from matplotlib import pyplot as plt
alphap = 1.5
probp = 0.75
expon = 2 * alphap - 1
slope = 0.76909

In [2]:
def betadist(alpha):
    """gives a random number from beta distribution"""
    return random.betavariate(alpha,alpha)

In [3]:
def decision(probability):
    """
    decides with a given probability whether to keep the right part
    """
    if float(probability) > random.random():
        return True
    else: 
        return False

In [4]:
def splitting(segment):
    """
    splits a given segment. left and right are endpoints of the segment
    returns : 
        xL -> length of the left segment
        xR -> length of the right segment
        flag -> keeping the right segment
        xLp, xRp -> probability(unnormalized) for being selected
        change -> change of normalization const
    """
    xL = segment * betadist(alphap)
    xR = segment - xL
    flag = decision(probp)
    xLp = xL**expon
    xRp = xR**expon
    change = xLp + xRp - segment**expon
    return xL, xR, flag, xLp, xRp, change

In [5]:
# def pickindex(frag_prob, frag_prob_sum):
#     """
#     picks up a segment to be subsequently split
#     """
#     r = random.uniform(0, frag_prob_sum)
#     sum_ = 0
#     for index in range(len(frag_prob)):
#         sum_ += frag_prob[index]
#         if sum_ < r:
#             continue
#         else:
#             return index

In [6]:
def pickindex(frag_prob, frac_sum):
    """
    picks up a segment to be subsequently split
    """
    r = random.uniform(0,1)
    
    if r <= frac_sum:
        sum_ = 0
        for index in range(len(frag_prob)):
            sum_ += frag_prob[index]
            if sum_ < r:
                continue
            else:
                return index

In [7]:
def number_length(segment_lengths,flags):
    
    N = 0
    M = 0

    for i in range(len(flags)):
        if flags[i]:
            N += 1
            M += segment_lengths[i]
            pass
        pass
    return N, M

In [8]:
fractal_dim = slope

def fractal_length(segment_lengths,flags):

    M_frac = 0

    for i in range(len(flags)):
        if flags[i]:
            M_frac += segment_lengths[i]**fractal_dim
            pass
        pass
    return M_frac

In [9]:
def realization_value(total_iteration, min_iteration, iteration_step, data_point = False):
    
    lengths = [1.]
    flags = [True]
    frag_prob = [1.] # raw probability, not normalized
    #frag_prob_sum = 1.0 # normalization const
    frac_sum = 1.0

    iteration_list = list(range(min_iteration, total_iteration + 1, iteration_step))
    
    M_realization = []
    
    for i in range(total_iteration + 1):
        
        #index = pickindex(frag_prob, frag_prob_sum)
        index = pickindex(frag_prob, frac_sum)
        
        if index != None and flags[index] == True:

            xL, xR, flag, xLp, xRp, change = splitting(lengths[index])
            
            lengths[index] = xL
            lengths.append(xR)
            flags.append(flag)
            frag_prob[index] = xLp 
            frag_prob.append(xRp)
            #frag_prob_sum += change
            frac_sum += change
            pass
        
        if i+1 in iteration_list:
            M_frac = fractal_length(lengths,flags)
            M_realization.append(M_frac)
        pass 
      
    if data_point:
        print(np.array(M_realization))
    
    else:
        M_frac = fractal_length(lengths,flags)
        return M_frac

In [10]:
def ensemble_average(total_iteration, ensemble_size):

    M_ensemble = 0
    
    for i in range(ensemble_size):
        M_list = realization_value(total_iteration, total_iteration, total_iteration)
        M_ensemble += M_list
        pass
    
    M_average = M_ensemble/ensemble_size
    
    print(M_average)

In [11]:
for i in range(5):
    realization_value(1_000_000, 100_000, 100_000, data_point = True)

[1.58051846 1.61057626 1.62810103 1.62231407 1.63331444 1.61860466
 1.62600879 1.63478103 1.62082068 1.628202  ]
[0.73227711 0.70980311 0.70427353 0.6927666  0.68698068 0.68487295
 0.6950179  0.69554601 0.69916453 0.69460427]
[1.21631528 1.25200043 1.26155942 1.2510224  1.25866919 1.25380906
 1.25258652 1.24871349 1.2267506  1.22910748]
[1.16050093 1.16318281 1.19041856 1.1878098  1.17658013 1.17493474
 1.1808113  1.17187246 1.15888894 1.1529558 ]
[1.65283314 1.63764172 1.6579269  1.68021959 1.6816629  1.69886346
 1.69962372 1.69114197 1.69612189 1.7042422 ]


In [12]:
for i in range(10):
    a_ = 1_000*(i+1)
    ensemble_average(a_, 100_000)

1.0014875640219525
1.001100330380423
1.0020998981319575
0.9986815732067508
0.9997045532577415
0.9995539277022972
1.0007992846587044
1.0012294776601212
1.0006029988461393
1.0009844895794249
